In [1]:
#imports
import scipy.io as sio
# import os
import sys
# import numpy as np
import pandas as pd
import time
from joblib import Parallel, delayed
from scipy import stats
# import matplotlib.pyplot as plt
import statsmodels.stats.multitest as multi
sys.path.append('/afs/dbic.dartmouth.edu/usr/wheatley/jd/')
from phaseScramble import *

In [2]:
#mark starting time
startTime = time.time()

#participant IDs
dbicIDs = np.array(["sid000007", "sid000009", "sid000560", "sid000535", "sid000102", "sid000416", "sid000499", "sid000142"])
cbsIDs = np.array(["hid000002", "hid000003", "hid000004", "hid000005", "hid000006", "hid000007", "hid000008", "hid000009"])

#pair numbers
pairNums = np.arange(2,len(dbicIDs)+2)

#make subject list data frame
subList = pd.DataFrame(np.transpose(np.tile(pairNums, (1, 2))),columns=['pairNum'])
subList['subID'] = np.concatenate((dbicIDs, cbsIDs), axis=0)
print(subList)

#get number of participants
numSubs = len(pairNums) * 2

#set number of permutations
permutations = 1000

#indicate whether or not we're debugging (if so, use a small subset of TRs and voxels to speed things up)
debug = True
debugTRs = 400
debugVox = 800

#set alpha for permutation tests
alpha = 0.05

#number of jobs for joblib
numJobs = 16

#set joblib verbosity -- don't go over 50 lest ye print like a million outputs and slow everything down
verbosity = 50

#select whether or not to save output
saveOutput = False

#set fitting distribution to normal
dist = getattr(stats, 'norm')

    pairNum      subID
0         2  sid000007
1         3  sid000009
2         4  sid000560
3         5  sid000535
4         6  sid000102
5         7  sid000416
6         8  sid000499
7         9  sid000142
8         2  hid000002
9         3  hid000003
10        4  hid000004
11        5  hid000005
12        6  hid000006
13        7  hid000007
14        8  hid000008
15        9  hid000009


In [3]:
#set data folder
folder = '/afs/dbic.dartmouth.edu/usr/wheatley/jd/control_tasks/'

#loop through participants...
boldData = [[]] * 2
dataShape = [[]] * 2 #******PART OF HACK! REMOVE LATER!*******
for TASK in [0,1]: #for each task... reading, then listening *********CHECK THIS!!!!***********

    #preallocate task data list
    boldData[TASK] = [[]] * numSubs
    dataShape[TASK] = [[]] * numSubs #******PART OF HACK! REMOVE LATER!*******

    for SUB in range(numSubs):

        #get file name
        fileName = folder + 'sub-' + subList['subID'][SUB] + '_ses-pair0' + str(subList['pairNum'][SUB]) + '_task-storytelling' + str(TASK + 3) + '_run-0' + str(TASK + 3) + '_bold_space-MNI152NLin2009cAsym_preproc_nuisRegr_2021.mat'

        #load data
        tmp = sio.loadmat(fileName) #load file
        boldData[TASK][SUB] = tmp['tseries'] #get timeseries data
        dataShape[TASK][SUB] = boldData[TASK][SUB].shape #******PART OF HACK! REMOVE LATER!*******

    #convert shape lists into dataframes and get minimum values for TRs and voxels across participants
    dataShape[TASK] = pd.DataFrame(dataShape[TASK], columns=['TRs','voxels'])

    #MAJORLY TEMPORARY HACK to deal with current differences in timeseries sizes
    #use smallest dimensions across participants (removing rows and columns from individual time series to get there)
    for SUB in range(numSubs):

        if debug:

            #take a small subset of each timeseries to speed up the debugging process
            boldData[TASK][SUB] = boldData[TASK][SUB][np.ix_(np.arange(debugTRs),range(debugVox))]

        else:

            #remove "excess" rows and columns
            for DIM in [0,1]: #TRs, voxels
                if dataShape[TASK].iloc[SUB,DIM] > dataShape[TASK].iloc[:,DIM].min():
                    numExtra = dataShape[TASK].iloc[SUB,DIM] - dataShape[TASK].iloc[:,DIM].min()
                    toRemove = np.array(range(dataShape[TASK].iloc[SUB,DIM]-numExtra,dataShape[TASK].iloc[SUB,DIM]))
                    boldData[TASK][SUB] = np.delete(boldData[TASK][SUB],toRemove,DIM)

In [4]:
#preallocate correlation lists
corrData = [[]] * 2
nullCorr = [[]] * 2
permTest = [[]] * 2

#map of what each participant-specific sublist in permTestMap contains
permTestMap = ['permPval','FDR_corrected_permPval','propSigFDRvoxels','normParams','KStest','badFits','badFitsSummary']

#Voxelwise correlation between participant and the rest of the group (mean)
for TASK in [0,1]: #for each task...

    corrData[TASK] = [[]] * numSubs
    nullCorr[TASK] = [[]] * numSubs
    permTest[TASK] = [[]] * numSubs

    def parallelSubWrapper(SUB,numSubs,boldData,permutations):
        """
        function to use with joblib below to run permutation tests for participants in parallel
        :param SUB: participant index
        :param numSubs: total number of participants in the analysis
        :param boldData: timeseries data
        :param permutations: number of permutations to use
        :return:
            corrData: real voxelwise correlations between participant and the mean across all other participants [voxels x 1]
            nullCorr: null voxelwise correlations between participant and the mean across all other participants [permutations x voxels]
            permTest: array containing various results from permutation test and normal fits to the null distribution (see permTestMap variable above and comments below)
        """

        #get mean of data from all participants EXCEPT the current participant
        otherSubs = np.arange(0,numSubs)
        otherSubs = np.delete(otherSubs,SUB)
        groupMean = np.mean([boldData[i] for i in otherSubs], axis=0)

        #get REAL correlation between current participant and groupMean
        corrData = fastColumnCorr(boldData[SUB], groupMean)

        #generate null correlations
        nullCorr = [[]] * permutations
        for PERM in range(permutations): #for each permutation...
            nullCorr[PERM] = fastColumnCorr(phase_scrambling(boldData[SUB]),groupMean)

        #convert permutation data to a permutations x voxels array
        nullCorr = np.asarray(nullCorr)

        #preallocate sublists of permTest array
        permTest = [[]] * len(permTestMap)

        #get permutation test p-values and apply FDR correction
        permTest[0] = [[]] * nullCorr.shape[1] #permutation test p-values
        for VOX in range(nullCorr.shape[1]):
            permTest[0][VOX] = len(np.where(abs(nullCorr[:,VOX]) > abs(corrData[VOX]))[0]) / float(permutations)
        permTest[1] = multi.fdrcorrection(permTest[0], alpha=alpha) #FDR correction
        permTest[2] = [[]] * 2 #preallocate lists for permutation test summary info
        permTest[2][0] = len(np.where(np.array(permTest[0]) < alpha)[0]) #proportion of voxels that show significant correlations with the group
        permTest[2][1] = len(np.where(permTest[1][0])[0]) / nullCorr.shape[1] #proportion of voxels that show significant FDR CORRECTED correlations with the group

        #fit normal distributions to null data
        permTest[3] = [[]] * nullCorr.shape[1] #preallocate list for voxelwise normal dist parameters
        permTest[4] = [[]] * nullCorr.shape[1] #preallocate list for voxelwise Kolmogorov–Smirnov test results
        permTest[5] = np.zeros((nullCorr.shape[1],), dtype=int) #preallocate voxelwise array to indicate bad fits
        for VOX in range(nullCorr.shape[1]): #for each voxel...
            permTest[3][VOX] = dist.fit(nullCorr[:,VOX]) #fit normal distribution
            permTest[4][VOX] = stats.kstest(nullCorr[:,VOX], "norm", permTest[3][VOX]) #measure goodness of fit
            if permTest[4][VOX][1] < alpha: #if a voxel has a bad normal fit...
                permTest[5][VOX] = 1 #flag it
        permTest[6] = pd.DataFrame(data={'numBadFits': [sum(permTest[5])], 'propBadFits': [sum(permTest[5]) / len(permTest[5])]}) #number and proportion of voxels with bad normal fits to the null distribution

        return corrData, nullCorr, permTest

    #run joblib
    tmp = Parallel(n_jobs=numJobs, verbose=verbosity)(delayed(parallelSubWrapper)(SUB,numSubs,boldData[TASK],permutations) for SUB in range(numSubs))

    #assign joblib outputs
    for SUB in range(numSubs):
        corrData[TASK][SUB] = tmp[SUB][0]
        nullCorr[TASK][SUB] = tmp[SUB][1]
        permTest[TASK][SUB] = tmp[SUB][2]

        #print some permutation test and goodness of fit summary info
        print('\nFinished processing participant ' + str(SUB + 1) + ' of ' + str(numSubs) + ' for task ' + str(TASK + 3))
        print('% voxels with FDR corrected significant correlation with group: ' + str((permTest[TASK][SUB][2][1]) * 100) + '%')
        print('% voxels for which null dist. was normal: ' + str((1 - permTest[TASK][SUB][6].iloc[0,1]) * 100) + '%')

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   1 tasks      | elapsed:   54.4s
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   54.8s remaining:  6.4min
[Parallel(n_jobs=16)]: Done   3 out of  16 | elapsed:   57.5s remaining:  4.2min
[Parallel(n_jobs=16)]: Done   4 out of  16 | elapsed:   57.7s remaining:  2.9min
[Parallel(n_jobs=16)]: Done   5 out of  16 | elapsed:   57.8s remaining:  2.1min
[Parallel(n_jobs=16)]: Done   6 out of  16 | elapsed:   57.8s remaining:  1.6min
[Parallel(n_jobs=16)]: Done   7 out of  16 | elapsed:   58.2s remaining:  1.2min
[Parallel(n_jobs=16)]: Done   8 out of  16 | elapsed:   58.2s remaining:   58.2s
[Parallel(n_jobs=16)]: Done   9 out of  16 | elapsed:   58.4s remaining:   45.4s
[Parallel(n_jobs=16)]: Done  10 out of  16 | elapsed:   58.6s remaining:   35.2s
[Parallel(n_jobs=16)]: Done  11 out of  16 | elapsed:   58.8s remaining:   26.7s
[Parallel(n_jobs=16)]: Done  12 out of  16 | elapse

In [5]:
#get group level null distributions for each voxel
if numSubs > 1: #if we're running the analysis on more than one participant...

    #preallocate lists for each task
    groupNull = [[]] * 2

    #for each task...
    for TASK in [0,1]:

        #preallocate sublists for groupNull array
        #groupNull[TASK][0] = raw null data concatenated across subs [permutations*numSubs x voxels]
        #groupNull[TASK][1] = fitted normal distribution parameters
        #groupNull[TASK][2] = Kolmogorov-Smirnov goodness of fit test results
        #groupNull[TASK][3] = indices of voxels with bad normal fits (KS test p-value < alpha)
        #groupNull[TASK][4] = number and proportion of voxels with bad normal fits to the group null distribution
        groupNull[TASK] = [[]] * 5

        #concatenate data from the first two participants to get things started
        groupNull[TASK][0] = np.concatenate((nullCorr[TASK][0],nullCorr[TASK][1]),axis=0)

        #concatenate data from any remaining participants
        if numSubs > 2: #if we're running the analysis on more than two participants...
            for SUB in range(2,numSubs): #for each participant...
                 groupNull[TASK][0] = np.concatenate((groupNull[TASK][0],nullCorr[TASK][SUB]),axis=0) #concatenate

        #fit normal distribution to group null
        groupNull[TASK][1] = [[]] * groupNull[TASK][0].shape[1] #preallocate list for each voxel
        groupNull[TASK][2] = [[]] * groupNull[TASK][0].shape[1] #preallocate list for each voxel
        groupNull[TASK][3] = np.zeros((groupNull[TASK][0].shape[1],), dtype=int)
        for VOX in range(groupNull[TASK][0].shape[1]): #for each voxel...
            groupNull[TASK][1][VOX] = dist.fit(groupNull[TASK][0][:,VOX]) #fit normal distribution
            groupNull[TASK][2][VOX] = stats.kstest(groupNull[TASK][0][:,VOX], "norm", groupNull[TASK][1][VOX]) #get KS goodness of fit
            if groupNull[TASK][2][VOX][1] < alpha:
                groupNull[TASK][3][VOX] = 1
        groupNull[TASK][4] = pd.DataFrame(data={'numBadFits': [sum(groupNull[TASK][3])], 'propBadFits': [sum(groupNull[TASK][3]) / len(groupNull[TASK][3])]}) #number and proportion of voxels with bad normal fits to the group null distribution

In [6]:
endTime = time.time()
duration = (endTime - startTime) / 60 #[min]
print('control tasks ISC duration: ' + str(duration))

control tasks ISC duration: 2.129042474428813


In [7]:
if saveOutput:
    import pickle
    with open(folder + 'controlISC.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([permTest, corrData, groupNull, duration], f)

In [8]:
# #what load syntax should look like:
# loadFile = folder + 'controlISC.pkl'
# with open(loadFile, 'rb') as f:
#     permTest, corrData, groupNull, duration = pickle.load(f)

In [9]:
# # Plot the PDF.
# mu = permTest[TASK][SUB][0][0]
# std = permTest[TASK][SUB][0][1]
# xmin, xmax = plt.xlim()
# x = np.linspace(xmin, xmax, 100)
# p = stats.norm.pdf(x, mu, std)
# plt.plot(x, p, 'k', linewidth=2)
# title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
# plt.title(title)